In [1]:
!pip install -r ../requirements.txt

In [2]:
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import pandas as pd
from bs4 import BeautifulSoup
import csv
import sys
# This is the website which we will be scraping
base_url = "https://www.cvedetails.com/"
s = requests.Session()

retries = Retry(total=100,
                backoff_factor=0.5,
                status_forcelist= [500, 502, 503, 504])

s.mount('https://', HTTPAdapter(max_retries=retries))

In [3]:
headers = ["#", "CVE ID", "CWE ID", "# of Exploits", "Vulnerability Type(s)", "Publish Date", "Update Date", "Score", "Gained Access Level", "Access", "Complexity", "Authentication", "Confidentiality", "Integrity", "Availability",  "Vendor", "Product"]
with open("vul_data.csv", "w") as f:
    write = csv.writer(f)
    write.writerow(headers)

def write_row(data):
    with open("vul_data.csv", "a") as f:
        writer = csv.writer(f)
        writer.writerow(data)
        

In [4]:
def scrape_table_vul(url):
    search_vul_page = BeautifulSoup(s.get(url).content, "html.parser")
    for row in search_vul_page.find_all(class_="srrowns"):
        # print(list(zip(headers, list(map(lambda x: x.get_text().strip(), row.find_all("td"))))))
        
        # Get the row of the table
        csv_row = list(map(lambda x: x.get_text().strip(), row.find_all("td")))

        # Add the description to the row
        

        # Get the Product and Vendor info and add it to the row
        cve_url = base_url + "/cve/" + csv_row[1]
        cve_page = BeautifulSoup(s.get(cve_url).content, "html.parser")
        if cve_page and cve_page.find(id="vulnversconuttable"):
            for x in cve_page.find(id="vulnversconuttable").find_all("td")[:2]:
                csv_row.append(x.get_text().strip())
        else:
            print(cve_url)
        
        write_row(csv_row)
        

In [5]:
def scrape_year_vul(url):
    year_vul_page = BeautifulSoup(s.get(url).content, "html.parser")
    pages = []
    for page_div in year_vul_page.find_all(class_="paging"):
        for page in page_div.find_all("a"):
            pages.append(base_url + page["href"])
    
    for page in pages:
        scrape_table_vul(url)
    
    

In [ ]:
def scrape_vul():
    
   
    # Get the urls for each of the years
    url = "https://www.cvedetails.com/browse-by-date.php"
    vul_page = BeautifulSoup(s.get(url).content, "html.parser")
    year_urls = []
    for i, year in enumerate(vul_page.find(class_="stats").find_all("th")):
        if year.find("a"):
            year_urls.append(base_url + year.find("a")["href"])
    # print(year_urls)
    
    # Go through each of the pages and add vulenerabilites to the table
    for year_url in year_urls:
        print(year_url)
        scrape_year_vul(year_url)

scrape_vul()

https://www.cvedetails.com//vulnerability-list/year-1999/vulnerabilities.html
https://www.cvedetails.com//vulnerability-list/year-2000/vulnerabilities.html
https://www.cvedetails.com//vulnerability-list/year-2001/vulnerabilities.html
https://www.cvedetails.com//vulnerability-list/year-2002/vulnerabilities.html
https://www.cvedetails.com//cve/CVE-2002-2413
https://www.cvedetails.com//cve/CVE-2002-2380
https://www.cvedetails.com//cve/CVE-2002-2413
https://www.cvedetails.com//cve/CVE-2002-2380
https://www.cvedetails.com//cve/CVE-2002-2413
https://www.cvedetails.com//cve/CVE-2002-2380
https://www.cvedetails.com//cve/CVE-2002-2413
https://www.cvedetails.com//cve/CVE-2002-2380
https://www.cvedetails.com//cve/CVE-2002-2413
https://www.cvedetails.com//cve/CVE-2002-2380
https://www.cvedetails.com//cve/CVE-2002-2413
https://www.cvedetails.com//cve/CVE-2002-2380
https://www.cvedetails.com//cve/CVE-2002-2413
https://www.cvedetails.com//cve/CVE-2002-2380
https://www.cvedetails.com//cve/CVE-2002-241

In [ ]:
vul = pd.read_csv("vul_data.csv")
print(vul.shape[0])